In [ ]:
import sys
import os
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

import pandas as pd
from isolated_ad_model.processing import PREPROCESS
import logging 
from AnomalyModelFactory import AnomalyModelFactory
from AnomalyRobustnessEvaluator import AnomalyRobustnessEvaluator

logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s [%(levelname)s] %(message)s', 
    handlers=[logging.StreamHandler()]
)

logger = logging.getLogger(__name__)

In [ ]:
#config
use_cached_model = True
model_name = "isolation_forest"
model_path = "model_" + model_name + ".pkl"

#obtain the dataset
dataset = pd.read_csv('../isolated_ad_model/ue.csv')
true_anomalies = dataset['Viavi.UE.anomalies']
ps = PREPROCESS(dataset)  # TODO: Is it possible to get rid of src/scale dependency? 
ps.process()
training_data = ps.data

#obtain the model
model_factory = AnomalyModelFactory() # TODO: Add configuration from file

if use_cached_model:
    model = model_factory.load_model(model_path) 
else:
    model = model_factory.build_model(training_data, true_anomalies, model_name)
    model_factory.save_model(model_path)

AnomalyRobustnessEvaluator._save_generated_dataset(training_data, './ue_base_processed')

#testing enviroment
evaluator = AnomalyRobustnessEvaluator(model)

pattern = (

    {
        "type": "interval",
        "features": None,
        "integer_features": None,
        "ratio": 0.1,
        "max_ratio": 0.3,
        "missing_value": 0.0,
        "probability": 0.6,
    },

)

In [ ]:
evaluator.test_a2pm(training_data, true_anomalies, pattern, save_adv_data=False)
evaluator.test_hsja(training_data, true_anomalies, save_adv_data=False)